In [4]:
import pandas as pd
import numpy as np
from xbbg import blp
from datetime import datetime, timedelta
import warnings
import time
warnings.filterwarnings('ignore')

class DefenseFirstDataPipeline:
    def __init__(self):
        """
        Defense First Strategy Data Pipeline - Fixed for xbbg MultiIndex issues
        Handles Bloomberg data retrieval, proxy stitching, and data quality validation
        """
        # Updated inception dates based on actual Bloomberg data availability
        self.assets = {
            'TLT': {
                'bloomberg': 'TLT US Equity',
                'inception_date': '2002-07-31',  # Updated to actual first data date
                'proxy_pre_inception': 'VUSTX US Equity',  # Vanguard Long-Term Treasury
                'proxy_alt': 'USGG30YR Index'  # 30-Year Treasury Yield (convert to price)
            },
            'GLD': {
                'bloomberg': 'GLD US Equity', 
                'inception_date': '2004-11-30',  # Updated to actual first data date
                'proxy_pre_inception': 'XAU Curncy',  # Gold Spot
                'proxy_alt': 'GOLDS Index'  # Gold Index
            },
            'DBC': {
                'bloomberg': 'DBC US Equity',
                'inception_date': '2006-02-28',  # Updated to actual first data date
                'proxy_pre_inception': 'BCOM Index',  # Bloomberg Commodity Index (switched to working proxy)
                'proxy_alt': 'CRB Index'  # CRB Commodity Index (now alternative)
            },
            'UUP': {
                'bloomberg': 'UUP US Equity',
                'inception_date': '2007-03-30',  # Updated to actual first data date
                'proxy_pre_inception': 'DXY Curncy',  # US Dollar Index
                'proxy_alt': 'USDINDEX Index'
            },
            'SPY': {
                'bloomberg': 'SPY US Equity',
                'inception_date': '1995-01-31',  # Updated to actual first data date (SPY had limited early data)
                'proxy_pre_inception': 'SPX Index',  # S&P 500 Index
                'proxy_alt': 'VFINX US Equity'  # Vanguard 500 Fund
            }
        }
        
        self.cash_proxy = 'US0003M Index'  # 90-day T-bills
        self.cash_alt = 'FEDL01 Index'     # Federal Funds Rate
        
    def get_complete_dataset(self, start_date='1986-01-01', end_date=None, frequency='M'):
        """
        Main method to retrieve complete historical dataset with proxy stitching
        """
        if end_date is None:
            end_date = datetime.now().strftime('%Y-%m-%d')
            
        print(f"Fetching Defense First dataset from {start_date} to {end_date}")
        
        # Initialize results container
        final_data = {}
        
        # Process each asset with proxy stitching
        for asset_name, asset_config in self.assets.items():
            print(f"Processing {asset_name}...")
            
            try:
                asset_data = self._get_asset_complete_history(
                    asset_name, asset_config, start_date, end_date, frequency
                )
                
                if not asset_data.empty:
                    final_data[asset_name] = asset_data
                    print(f"  ✓ {asset_name}: {len(asset_data)} observations from {asset_data.index[0]} to {asset_data.index[-1]}")
                else:
                    print(f"  ✗ {asset_name}: No data retrieved")
                    final_data[asset_name] = pd.Series()
                    
            except Exception as e:
                print(f"  ✗ Error processing {asset_name}: {e}")
                final_data[asset_name] = pd.Series()
        
        # Get cash proxy data
        print("Processing cash proxy (90-day T-bills)...")
        try:
            cash_data = self._get_cash_data(start_date, end_date, frequency)
            final_data['CASH'] = cash_data
            if not cash_data.empty:
                print(f"  ✓ CASH: {len(cash_data)} observations")
            else:
                print(f"  ✗ CASH: No data retrieved")
        except Exception as e:
            print(f"  ✗ Error processing cash data: {e}")
            final_data['CASH'] = pd.Series()
        
        # Create date range for alignment
        date_range = pd.date_range(start=start_date, end=end_date, freq='M')
        
        # Combine all data into single DataFrame with proper alignment
        combined_data = pd.DataFrame(index=date_range)
        for asset_name, asset_series in final_data.items():
            if not asset_series.empty:
                combined_data[asset_name] = asset_series.reindex(date_range, method='ffill')
            else:
                combined_data[asset_name] = np.nan
        
        # Data quality validation
        quality_report = self._validate_data_quality(combined_data)
        print("\n=== DATA QUALITY REPORT ===")
        self._print_quality_report(quality_report)
        
        return combined_data
    
    def _get_asset_complete_history(self, asset_name, asset_config, start_date, end_date, frequency):
        """
        Get complete history for single asset with proxy stitching and fallback logic
        """
        inception_date = asset_config['inception_date']
        bloomberg_ticker = asset_config['bloomberg']
        proxy_ticker = asset_config['proxy_pre_inception']
        proxy_alt = asset_config.get('proxy_alt', '')
        
        # Determine if we need proxy data
        if pd.to_datetime(start_date) < pd.to_datetime(inception_date):
            # Need proxy data for pre-inception period
            print(f"    Fetching proxy data for {asset_name} pre-{inception_date}...")
            
            # Try primary proxy first
            proxy_data = self._fetch_single_series(
                proxy_ticker, start_date, inception_date, frequency
            )
            
            # If primary proxy fails and alternative exists, try alternative
            if proxy_data.empty and proxy_alt:
                print(f"    Primary proxy failed, trying alternative: {proxy_alt}...")
                proxy_data = self._fetch_single_series(
                    proxy_alt, start_date, inception_date, frequency
                )
            
            # Get ETF data from inception forward
            print(f"    Fetching ETF data for {asset_name} from {inception_date}...")
            etf_data = self._fetch_single_series(
                bloomberg_ticker, inception_date, end_date, frequency
            )
            
            # Stitch data together
            if not proxy_data.empty and not etf_data.empty:
                complete_data = self._stitch_proxy_to_etf(proxy_data, etf_data, inception_date)
            elif not etf_data.empty:
                complete_data = etf_data
            elif not proxy_data.empty:
                complete_data = proxy_data
            else:
                complete_data = pd.Series()
            
        else:
            # Only need ETF data
            complete_data = self._fetch_single_series(
                bloomberg_ticker, start_date, end_date, frequency
            )
        
        return complete_data
    
    def _fetch_single_series(self, ticker, start_date, end_date, frequency, retries=3):
        """
        Fetch single Bloomberg series with proper xbbg handling
        """
        for attempt in range(retries):
            try:
                # Fetch data - handle both total return and price
                print(f"      Attempting to fetch {ticker}...")
                
                # Try total return first
                try:
                    data = blp.bdh(
                        tickers=[ticker],  # Use list to ensure consistent structure
                        flds=['TOT_RETURN_INDEX_GROSS_DVDS'],
                        start_date=start_date,
                        end_date=end_date,
                        Per=frequency,
                        CshAdjNormal=True,
                        CshAdjAbnormal=True,
                        CapChg=True
                    )
                    
                    if not data.empty and len(data.columns) > 0:
                        # Handle MultiIndex columns properly
                        if isinstance(data.columns, pd.MultiIndex):
                            # Get first non-null column
                            for col in data.columns:
                                series = data[col].dropna()
                                if not series.empty:
                                    return series
                        else:
                            return data.iloc[:, 0].dropna()
                            
                except Exception:
                    # Fall back to price data
                    data = blp.bdh(
                        tickers=[ticker],
                        flds=['PX_LAST'],
                        start_date=start_date,
                        end_date=end_date,
                        Per=frequency
                    )
                    
                    if not data.empty and len(data.columns) > 0:
                        if isinstance(data.columns, pd.MultiIndex):
                            for col in data.columns:
                                series = data[col].dropna()
                                if not series.empty:
                                    return series
                        else:
                            return data.iloc[:, 0].dropna()
                
                # If still no data, try alternative approach
                data = blp.bdh(ticker, 'PX_LAST', start_date, end_date)
                if not data.empty:
                    return data.iloc[:, 0].dropna() if len(data.columns) > 0 else pd.Series()
                    
            except Exception as e:
                print(f"      Attempt {attempt + 1} failed for {ticker}: {str(e)[:100]}...")
                if attempt < retries - 1:
                    time.sleep(1 + attempt)  # Progressive backoff
        
        print(f"      Failed to retrieve data for {ticker} after {retries} attempts")
        return pd.Series()
    
    def _stitch_proxy_to_etf(self, proxy_data, etf_data, splice_date):
        """
        Stitch proxy data to ETF data at splice point with improved scaling
        """
        if proxy_data.empty and etf_data.empty:
            return pd.Series()
        elif proxy_data.empty:
            return etf_data
        elif etf_data.empty:
            return proxy_data
        
        splice_date = pd.to_datetime(splice_date)
        
        # Get pre-splice proxy data
        pre_splice = proxy_data[proxy_data.index < splice_date].copy()
        
        if pre_splice.empty:
            return etf_data
        
        # Find best scaling approach
        etf_start_value = etf_data.iloc[0] if not etf_data.empty else None
        proxy_end_value = pre_splice.iloc[-1] if not pre_splice.empty else None
        
        if etf_start_value and proxy_end_value and proxy_end_value != 0:
            # Method 1: Direct scaling
            scale_factor = etf_start_value / proxy_end_value
            scaled_proxy = pre_splice * scale_factor
            
            # Combine scaled proxy with ETF data
            combined = pd.concat([scaled_proxy, etf_data])
            return combined.sort_index()
        else:
            # Method 2: Normalize both series to 100 at splice point
            if not pre_splice.empty and not etf_data.empty:
                proxy_normalized = (pre_splice / pre_splice.iloc[-1]) * 100
                etf_normalized = (etf_data / etf_data.iloc[0]) * 100
                
                combined = pd.concat([proxy_normalized, etf_normalized])
                return combined.sort_index()
        
        # Fallback: just concatenate
        combined = pd.concat([pre_splice, etf_data])
        return combined.sort_index()
    
    def _get_cash_data(self, start_date, end_date, frequency):
        """
        Get cash proxy data with improved yield-to-price conversion and fallback logic
        """
        try:
            # Try primary cash proxy (3-month Treasury)
            cash_series = self._fetch_single_series(
                self.cash_proxy, start_date, end_date, frequency
            )
            
            if cash_series.empty:
                # Try alternative (Federal Funds Rate)
                print("    Trying alternative cash proxy (Fed Funds)...")
                cash_series = self._fetch_single_series(
                    self.cash_alt, start_date, end_date, frequency
                )
            
            if not cash_series.empty:
                # Convert yield to cumulative return index
                # Assuming yields are in percentage terms
                monthly_returns = cash_series / 100 / 12  # Convert annual % to monthly
                monthly_returns = monthly_returns.fillna(method='ffill').fillna(0)
                
                # Build cumulative return index starting at 100
                cash_index = (1 + monthly_returns).cumprod() * 100
                return cash_index
            
        except Exception as e:
            print(f"    Error in cash data processing: {e}")
        
        # Fallback: create flat cash series at 100
        print("    Using fallback flat cash series")
        date_range = pd.date_range(start=start_date, end=end_date, freq='M')
        return pd.Series(100.0, index=date_range, name='CASH')
    
    def _validate_data_quality(self, data):
        """
        Comprehensive data quality validation
        """
        quality_report = {}
        
        for column in data.columns:
            series = data[column]
            
            quality_report[column] = {
                'total_observations': len(series),
                'missing_count': series.isnull().sum(),
                'missing_percentage': series.isnull().mean() * 100,
                'zero_values': (series == 0).sum(),
                'negative_values': (series < 0).sum() if series.dtype in ['float64', 'int64'] else 0,
                'first_valid_date': series.first_valid_index(),
                'last_valid_date': series.last_valid_index(),
                'data_range': (series.min(), series.max()) if not series.empty and series.notna().any() else (None, None),
                'suspicious_gaps': self._detect_suspicious_gaps(series),
                'data_coverage': (len(series) - series.isnull().sum()) / len(series) * 100
            }
        
        return quality_report
    
    def _detect_suspicious_gaps(self, series, max_gap_days=65):
        """
        Detect suspicious data gaps longer than threshold (adjusted for monthly data)
        """
        if series.empty or series.isnull().all():
            return []
        
        # Find gaps in data
        valid_dates = series.dropna().index
        if len(valid_dates) < 2:
            return []
        
        gaps = []
        for i in range(1, len(valid_dates)):
            gap_days = (valid_dates[i] - valid_dates[i-1]).days
            if gap_days > max_gap_days:  # More than ~2 months for monthly data
                gaps.append({
                    'start': valid_dates[i-1],
                    'end': valid_dates[i], 
                    'days': gap_days
                })
        
        return gaps
    
    def _print_quality_report(self, quality_report):
        """
        Print formatted data quality report
        """
        for asset, metrics in quality_report.items():
            coverage = metrics['data_coverage']
            if coverage > 90:
                status = "✓"
            elif coverage > 50:
                status = "⚠"
            else:
                status = "✗"
                
            print(f"\n{status} {asset}:")
            print(f"  Observations: {metrics['total_observations']}")
            print(f"  Coverage: {coverage:.1f}% ({metrics['total_observations'] - metrics['missing_count']} valid)")
            
            if metrics['first_valid_date'] and metrics['last_valid_date']:
                print(f"  Date Range: {metrics['first_valid_date'].strftime('%Y-%m')} to {metrics['last_valid_date'].strftime('%Y-%m')}")
                
                if metrics['data_range'][0] is not None:
                    print(f"  Value Range: {metrics['data_range'][0]:.2f} to {metrics['data_range'][1]:.2f}")
            
            if metrics['suspicious_gaps']:
                print(f"  ⚠ Data Gaps: {len(metrics['suspicious_gaps'])} gaps > 65 days")
    
    def save_data_to_files(self, data, base_filename='defense_first_data'):
        """
        Save processed data with proper datetime index formatting - Fixed Excel writer
        """
        # Remove completely empty columns
        data_clean = data.dropna(axis=1, how='all')
        
        # Ensure index is properly named and timezone-naive
        data_clean.index.name = 'Date'
        if hasattr(data_clean.index, 'tz') and data_clean.index.tz is not None:
            data_clean.index = data_clean.index.tz_localize(None)
        
        # CSV format with proper date formatting
        data_clean.to_csv(f'{base_filename}.csv', date_format='%Y-%m-%d')
        print(f"Data saved to {base_filename}.csv (Date as datetime index)")
        
        # Parquet format preserves datetime index automatically
        data_clean.to_parquet(f'{base_filename}.parquet')
        print(f"Data saved to {base_filename}.parquet (datetime index preserved)")
        
        # Excel format with multiple sheets - fixed writer
        try:
            with pd.ExcelWriter(f'{base_filename}.xlsx', engine='openpyxl') as writer:
                # Main data
                data_clean.to_excel(writer, sheet_name='Price_Data')
                
                # Quality metrics
                quality_df = pd.DataFrame(self._validate_data_quality(data)).T
                quality_df.to_excel(writer, sheet_name='Quality_Metrics')
                
                # Returns calculation
                returns = data_clean.pct_change().dropna()
                if not returns.empty:
                    returns.index.name = 'Date'
                    if hasattr(returns.index, 'tz') and returns.index.tz is not None:
                        returns.index = returns.index.tz_localize(None)
                    returns.to_excel(writer, sheet_name='Monthly_Returns')
                
                # Summary statistics
                summary_stats = data_clean.describe()
                summary_stats.to_excel(writer, sheet_name='Summary_Stats')
                
                # Metadata
                metadata = pd.DataFrame({
                    'Dataset_Info': [
                        f'Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
                        f'Frequency: Monthly',
                        f'Start_Date: {data_clean.index.min().strftime("%Y-%m-%d")}',
                        f'End_Date: {data_clean.index.max().strftime("%Y-%m-%d")}',
                        f'Total_Observations: {len(data_clean)}',
                        f'Assets: {", ".join(data_clean.columns)}',
                        'Strategy: Defense First Tactical Allocation'
                    ]
                })
                metadata.to_excel(writer, sheet_name='Metadata', index=False)
            
            print(f"Data saved to {base_filename}.xlsx (datetime index with proper formatting)")
            
        except Exception as e:
            print(f"Warning: Excel save failed ({e}), but CSV and Parquet saved successfully")
        
        # Create a data loading helper function file
        self._create_data_loader_helper(base_filename)

    def _create_data_loader_helper(self, base_filename):
        """
        Create a helper Python file for properly loading the saved data
        """
        helper_code = f'''
import pandas as pd
import numpy as np
from datetime import datetime

def load_defense_first_data(file_format='parquet', file_path='{base_filename}'):
    """
    Load Defense First dataset with proper datetime index
    
    Parameters:
    file_format: 'csv', 'parquet', or 'excel'
    file_path: base filename (without extension)
    
    Returns:
    DataFrame with datetime index
    """
    
    if file_format.lower() == 'csv':
        # Load CSV with proper datetime parsing
        data = pd.read_csv(
            f'{{file_path}}.csv',
            index_col=0,
            parse_dates=True
        )
        
    elif file_format.lower() == 'parquet':
        # Load Parquet (datetime index preserved automatically)
        data = pd.read_parquet(f'{{file_path}}.parquet')
        
    elif file_format.lower() == 'excel':
        # Load Excel with proper datetime parsing
        data = pd.read_excel(
            f'{{file_path}}.xlsx',
            sheet_name='Price_Data',
            index_col=0,
            parse_dates=True
        )
        
    else:
        raise ValueError("file_format must be 'csv', 'parquet', or 'excel'")
    
    # Ensure index is datetime and properly named
    data.index = pd.to_datetime(data.index)
    data.index.name = 'Date'
    
    # Sort by date to ensure proper chronological order
    data = data.sort_index()
    
    return data

def load_returns_data(file_path='{base_filename}'):
    """Load monthly returns with datetime index"""
    try:
        returns = pd.read_excel(
            f'{{file_path}}.xlsx',
            sheet_name='Monthly_Returns',
            index_col=0,
            parse_dates=True
        )
        returns.index = pd.to_datetime(returns.index)
        returns.index.name = 'Date'
        return returns.sort_index()
    except Exception as e:
        print(f"Could not load returns from Excel: {{e}}")
        # Calculate returns from main data
        data = load_defense_first_data('parquet', file_path)
        returns = data.pct_change().dropna()
        return returns

def get_data_summary(file_path='{base_filename}'):
    """Get comprehensive data summary"""
    
    # Load main data
    data = load_defense_first_data('parquet', file_path)
    
    summary = {{
        'shape': data.shape,
        'date_range': (data.index.min(), data.index.max()),
        'frequency': pd.infer_freq(data.index),
        'assets': list(data.columns),
        'missing_data': data.isnull().sum().to_dict(),
        'data_coverage': ((len(data) - data.isnull().sum()) / len(data) * 100).to_dict()
    }}
    
    return summary

# Example usage:
if __name__ == "__main__":
    # Load data
    data = load_defense_first_data('parquet')  # Recommended format
    print(f"Loaded data shape: {{data.shape}}")
    print(f"Date range: {{data.index.min()}} to {{data.index.max()}}")
    print(f"Assets: {{', '.join(data.columns)}}")
    
    # Display sample
    print("\\nSample data (last 5 observations):")
    print(data.tail())
    
    # Load returns
    returns = load_returns_data()
    print(f"\\nReturns data shape: {{returns.shape}}")
    
    # Get summary
    summary = get_data_summary()
    print(f"\\nData Summary:")
    for key, value in summary.items():
        print(f"  {{key}}: {{value}}")
'''
        
        with open(f'{base_filename}_loader.py', 'w') as f:
            f.write(helper_code)
        
        print(f"Data loader helper saved to {base_filename}_loader.py")

    def validate_datetime_index(self, data):
        """
        Validate that datetime index is properly formatted
        """
        checks = {
            'is_datetime_index': isinstance(data.index, pd.DatetimeIndex),
            'is_sorted': data.index.is_monotonic_increasing,
            'has_duplicates': data.index.duplicated().any(),
            'frequency': pd.infer_freq(data.index),
            'timezone_aware': data.index.tz is not None,
            'index_name': data.index.name
        }
        
        print("\n=== DATETIME INDEX VALIDATION ===")
        for check, result in checks.items():
            status = "✓" if (result if check != 'has_duplicates' else not result) else "✗"
            print(f"{status} {check}: {result}")
        
        return all([
            checks['is_datetime_index'],
            checks['is_sorted'], 
            not checks['has_duplicates']
        ])
    
    def check_bloomberg_connectivity(self):
        """
        Test Bloomberg connection with xbbg specific tests
        """
        try:
            # Simple connectivity test
            test_data = blp.bdh('SPY US Equity', 'PX_LAST', '2020-01-01', '2020-01-31')
            if not test_data.empty:
                print("✓ Bloomberg xbbg connectivity successful")
                return True
            else:
                print("✗ Bloomberg returned empty data")
                return False
        except Exception as e:
            print(f"✗ Bloomberg connectivity issue: {e}")
            return False
    
    def test_individual_tickers(self):
        """
        Test each ticker individually to identify specific issues
        """
        print("\n=== INDIVIDUAL TICKER TESTING ===")
        
        all_tickers = []
        for asset_config in self.assets.values():
            all_tickers.extend([
                asset_config['bloomberg'],
                asset_config['proxy_pre_inception'],
                asset_config.get('proxy_alt', '')
            ])
        all_tickers.append(self.cash_proxy)
        all_tickers = [t for t in all_tickers if t]  # Remove empty strings
        
        results = {}
        for ticker in set(all_tickers):  # Remove duplicates
            try:
                test_data = self._fetch_single_series(ticker, '2020-01-01', '2020-03-31', 'M')
                if not test_data.empty:
                    results[ticker] = f"✓ {len(test_data)} observations"
                else:
                    results[ticker] = "✗ No data retrieved"
            except Exception as e:
                results[ticker] = f"✗ Error: {str(e)[:50]}"
        
        for ticker, result in results.items():
            print(f"  {ticker}: {result}")
        
        return results

# Updated main execution block
if __name__ == "__main__":
    # Initialize data pipeline
    pipeline = DefenseFirstDataPipeline()
    
    # Check Bloomberg connectivity
    if not pipeline.check_bloomberg_connectivity():
        print("Bloomberg connection failed - check xbbg setup")
        exit(1)
    
    # Test individual tickers
    pipeline.test_individual_tickers()
    
    # Get complete dataset
    print("\n=== FETCHING COMPLETE DATASET ===")
    data = pipeline.get_complete_dataset(
        start_date='1986-01-01', 
        end_date='2025-06-30',
        frequency='M'
    )
    
    if not data.empty and not data.dropna(axis=1, how='all').empty:
        clean_data = data.dropna(axis=1, how='all')
        
        # Validate datetime index
        is_valid_index = pipeline.validate_datetime_index(clean_data)
        
        print(f"\nDataset shape: {clean_data.shape}")
        print(f"Date range: {clean_data.index.min().strftime('%Y-%m-%d')} to {clean_data.index.max().strftime('%Y-%m-%d')}")
        print(f"Frequency: {pd.infer_freq(clean_data.index)}")
        print(f"Assets with data: {', '.join(clean_data.columns[clean_data.notna().any()])}")
        
        # Save processed data with proper datetime formatting
        pipeline.save_data_to_files(clean_data)
        
        # Display sample data with formatted dates
        print("\n=== SAMPLE DATA (Last 5 observations) ===")
        sample_data = clean_data.tail()
        sample_data.index = sample_data.index.strftime('%Y-%m-%d')
        print(sample_data)
        
        # Display first valid data points with formatted dates
        print("\n=== FIRST VALID DATA POINTS ===")
        for col in clean_data.columns:
            first_valid = clean_data[col].first_valid_index()
            if first_valid:
                first_val = clean_data.loc[first_valid, col]
                print(f"{col}: {first_valid.strftime('%Y-%m-%d')} = {first_val:.2f}")
        
        print(f"\n✓ All files saved with proper datetime index formatting")
        print(f"✓ Data loader helper created: defense_first_data_loader.py")
        
    else:
        print("Failed to retrieve any valid data")

✓ Bloomberg xbbg connectivity successful

=== INDIVIDUAL TICKER TESTING ===
      Attempting to fetch UUP US Equity...
      Attempting to fetch VUSTX US Equity...
      Attempting to fetch SPY US Equity...
      Attempting to fetch USGG30YR Index...
      Attempting to fetch VFINX US Equity...
      Attempting to fetch TLT US Equity...
      Attempting to fetch DXY Curncy...
      Attempting to fetch GLD US Equity...
      Attempting to fetch US0003M Index...
      Attempting to fetch SPX Index...
      Attempting to fetch GOLDS Index...
      Attempting to fetch CRB Index...
      Attempting to fetch CRB Index...
      Attempting to fetch CRB Index...
      Failed to retrieve data for CRB Index after 3 attempts
      Attempting to fetch DBC US Equity...
      Attempting to fetch XAU Curncy...
      Attempting to fetch USDINDEX Index...
      Attempting to fetch USDINDEX Index...
      Attempting to fetch USDINDEX Index...
      Failed to retrieve data for USDINDEX Index after 3 attem